In [1]:
from langchain import OpenAI, LLMChain
from langchain.memory import ConversationBufferMemory
# from langchain.retrievers import FaissRetriever
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import streamlit as st
import os
import openai
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.callbacks import get_openai_callback
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain
import json 
from langchain.memory import VectorStoreRetrieverMemory,ConversationBufferMemory, ConversationEntityMemory, ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain


# Specify the directory for persistent storage
persist_directory = os.path.dirname(os.getcwd()) + '\\data\\chroma\\vietcap_reports'
embedding = OpenAIEmbeddings()
# Initialize Chroma with embeddings and directory
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

#prompt template
sysmessage = """
You are a company information extractor and also an expert in Financial investment.
You will receive question from users, there are steps that you will follow to answer the questions:
First step (just think, don't write): you separate the question into sub-questions 
Second step (just think, don't write): Give detailed answers with numbers for evidence for each sub-question in a 2-level structure, level 1 has 3 headings in numbered list, each heading in level 1 then is explained in 3 bullet points. 
Third step (now write): you show level 1 headings, then you show level 1 headings + their bullet points, then show 1 key takeaway. Remember that you must always give numbers when you compare anything.
Finally generate 3 related questions that users might be interested in.
Important instruction: Answer I don't know if the information is not in the context.
"""


# Initialize OpenAI embeddings
openai.api_key = os.environ.get('OPENAI_API_KEY')

# Initialize ChatOpenAI with temperature set to 0
llm = ChatOpenAI(temperature=0,
                    model = 'gpt-3.5-turbo'
                    #streaming=True
                    )


# Initialize the retriever with your FAISS vector database
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# Create the conversation chain
# conversation_chain = ConversationChain(
#     llm=llm,
#     prompt=prompt,
#     memory=memory,
#     # retriever=retriever
# )

template = sysmessage+"Based on the following context to answer the questions\n{context}\nand chat history\n{chat_history}\n### Question:\n{question}\n\n### Answer:"
prompt = PromptTemplate(template=template, input_variables=["question"])
template_short = "History:\n{chat_history}\n### Question:\n{question}\n\n### Answer:"
prompt_short = PromptTemplate(template=template_short, input_variables=["question"])


qa_chain = ConversationalRetrievalChain.from_llm(llm=llm,
                                      retriever=retriever,
                                       return_source_documents=False,verbose=True,
                                 memory = memory,combine_docs_chain_kwargs={'prompt': prompt},
     condense_question_prompt=prompt_short,
                                        )

In [2]:
question_1 = "general information of VNM (company name, industry, main product) and its historical net revenue and profit in 2022 and 2023, quickly comment on the trend"
question_2 = "elaborate on the 1st point you mentioned" 
question_3 = "answer the 1st related question"

In [3]:
answer_1=qa_chain({"question": question_1})

d:\Data science\Python materials\Financial-GPT\Financial GPT_2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


: 

In [9]:
print('\nQuestion: ' + answer_1['question'])
print('\nAnswer:\n'+answer_1['answer'])


Question: general information of VNM (company name, industry, main product) and its historical net revenue and profit in 2022 and 2023, quickly comment on the trend

Answer:
1. General Information of VNM:
   - Company Name: VNM
   - Industry: Food and Beverage
   - Main Product: Dairy products

2. Historical Net Revenue and Profit in 2022 and 2023:
   - 2022 Net Revenue: VND 60,369 billion
   - 2023 Net Revenue: VND 63,468 billion
   - 2022 Gross Profit: VND 24,545 billion
   - 2023 Gross Profit: VND 26,462 billion

3. Key Takeaway:
   - The company VNM experienced an increase in both net revenue and gross profit from 2022 to 2023, indicating a positive trend in its financial performance.

### Related Questions:
1. What factors contributed to the increase in VNM's net revenue and gross profit from 2022 to 2023?
2. How does VNM's financial performance in 2023 compare to other companies in the food and beverage industry?
3. What are the growth prospects for VNM in the medium to long ter

In [10]:
#print keys in answer_1['chat_history'] list
print('\nChat history: ')
for key in answer_1['chat_history']:
    print(key)



Chat history: 
content='general information of VNM (company name, industry, main product) and its historical net revenue and profit in 2022 and 2023, quickly comment on the trend'
content="1. General Information of VNM:\n   - Company Name: VNM\n   - Industry: Food and Beverage\n   - Main Product: Dairy products\n\n2. Historical Net Revenue and Profit in 2022 and 2023:\n   - 2022 Net Revenue: VND 60,369 billion\n   - 2023 Net Revenue: VND 63,468 billion\n   - 2022 Gross Profit: VND 24,545 billion\n   - 2023 Gross Profit: VND 26,462 billion\n\n3. Key Takeaway:\n   - The company VNM experienced an increase in both net revenue and gross profit from 2022 to 2023, indicating a positive trend in its financial performance.\n\n### Related Questions:\n1. What factors contributed to the increase in VNM's net revenue and gross profit from 2022 to 2023?\n2. How does VNM's financial performance in 2023 compare to other companies in the food and beverage industry?\n3. What are the growth prospects f

In [11]:
answer_2=qa_chain({"question": question_2})




> Entering new LLMChain chain...
Prompt after formatting:
History:

Human: general information of VNM (company name, industry, main product) and its historical net revenue and profit in 2022 and 2023, quickly comment on the trend
Assistant: 1. General Information of VNM:
   - Company Name: VNM
   - Industry: Food and Beverage
   - Main Product: Dairy products

2. Historical Net Revenue and Profit in 2022 and 2023:
   - 2022 Net Revenue: VND 60,369 billion
   - 2023 Net Revenue: VND 63,468 billion
   - 2022 Gross Profit: VND 24,545 billion
   - 2023 Gross Profit: VND 26,462 billion

3. Key Takeaway:
   - The company VNM experienced an increase in both net revenue and gross profit from 2022 to 2023, indicating a positive trend in its financial performance.

### Related Questions:
1. What factors contributed to the increase in VNM's net revenue and gross profit from 2022 to 2023?
2. How does VNM's financial performance in 2023 compare to other companies in the food and beverage industry

In [12]:
print('\nQuestion: ' + answer_2['question'])
# print('\nHistory'+ answer_1['chat_history'])
print('\nAnswer:\n'+answer_2['answer'])


Question: what is its main product and how does it contribute to the company's revenue?

Answer:
1. General Information of VNM:
   - Company Name: VNM
   - Industry: Food and Beverage
   - Main Product: Dairy products

2. Historical Net Revenue and Profit in 2022 and 2023:
   - 2022 Net Revenue: VND 60,369 billion
   - 2023 Net Revenue: VND 63,468 billion
   - 2022 Gross Profit: VND 24,545 billion
   - 2023 Gross Profit: VND 26,462 billion

3. Key Takeaway:
   - The company VNM experienced an increase in both net revenue and gross profit from 2022 to 2023, indicating a positive trend in its financial performance.

### Related Questions:
1. What factors contributed to the increase in VNM's net revenue and gross profit from 2022 to 2023?
2. How does VNM's financial performance in 2023 compare to other companies in the food and beverage industry?
3. What are the growth prospects for VNM in the medium to long term based on its historical financial data?


In [13]:
answer_3=qa_chain({"question": question_3})




> Entering new LLMChain chain...
Prompt after formatting:
History:

Human: general information of VNM (company name, industry, main product) and its historical net revenue and profit in 2022 and 2023, quickly comment on the trend
Assistant: 1. General Information of VNM:
   - Company Name: VNM
   - Industry: Food and Beverage
   - Main Product: Dairy products

2. Historical Net Revenue and Profit in 2022 and 2023:
   - 2022 Net Revenue: VND 60,369 billion
   - 2023 Net Revenue: VND 63,468 billion
   - 2022 Gross Profit: VND 24,545 billion
   - 2023 Gross Profit: VND 26,462 billion

3. Key Takeaway:
   - The company VNM experienced an increase in both net revenue and gross profit from 2022 to 2023, indicating a positive trend in its financial performance.

### Related Questions:
1. What factors contributed to the increase in VNM's net revenue and gross profit from 2022 to 2023?
2. How does VNM's financial performance in 2023 compare to other companies in the food and beverage industry

In [14]:
print('\nQuestion: ' + answer_3['question'])
# print('\nHistory'+ answer_1['chat_history'])
print('\nAnswer:\n'+answer_3['answer'])


Question: answer the 1st related question

Answer:
1. Factors Contributing to the Increase in VNM's Net Revenue and Gross Profit from 2022 to 2023:
   - Increased demand for dairy products: Higher consumer demand for VNM's dairy products likely led to an increase in sales revenue.
   - Effective cost management strategies: Efficient cost control measures may have helped improve gross profit margins for VNM.
   - Successful marketing campaigns: Marketing efforts that resonated with consumers could have driven higher sales and revenue for the company.

2. Comparison of VNM's Financial Performance in 2023 with Other Companies in the Food and Beverage Industry:
   - VNM's revenue and profit growth in 2023 outperformed many other companies in the food and beverage industry.
   - The company's gross profit margin improvement of 80bps in 2023 indicates strong operational performance.
   - VNM's market share and brand recognition in the dairy sector position it favorably compared to competito